In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import torch
import torchvision
import torch.nn as nn
import numpy as np

import sys
sys.path.append("../src/")
import model

In [25]:
vgg = torchvision.models.vgg11()

In [30]:
vgg.features

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [33]:
convNet = model.convNet()

In [34]:
convNet

convNet(
  (features): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=800, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=10, bias=True)
  )
)

In [35]:
for index, layer in enumerate(convNet.features):
    print(index, layer)

0 Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
1 ReLU()
2 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
3 Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
4 ReLU()
5 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [31]:
convNet[0]

TypeError: 'convNet' object does not support indexing

In [7]:
mnist = torchvision.datasets.MNIST("../data/", train=True)

In [41]:
arr = np.array(mnist[0][0])

In [51]:
arr.shape

(28, 28)

In [54]:
arr_ = np.expand_dims(arr, axis=2)

In [55]:
arr_.shape

(28, 28, 1)

In [57]:
arr_..shape

(28, 28)

In [19]:
type(mnist[0][0])

PIL.Image.Image

In [10]:
mnist[1]

(<PIL.Image.Image image mode=L size=28x28 at 0x13CBA3588>, 0)

In [11]:
mnist[2]

(<PIL.Image.Image image mode=L size=28x28 at 0x13CBA37B8>, 4)

In [12]:
mnist[3]

(<PIL.Image.Image image mode=L size=28x28 at 0x13CBA3630>, 1)

In [13]:
mnist[4]

(<PIL.Image.Image image mode=L size=28x28 at 0x13CBA3550>, 9)

In [14]:
mnist[5]

(<PIL.Image.Image image mode=L size=28x28 at 0x13CBA39B0>, 2)

In [15]:
mnist[6]

(<PIL.Image.Image image mode=L size=28x28 at 0x13CBA3A58>, 1)

In [16]:
mnist[7]

(<PIL.Image.Image image mode=L size=28x28 at 0x13CBA3B00>, 3)

In [17]:
mnist[8]

(<PIL.Image.Image image mode=L size=28x28 at 0x13CBA3BA8>, 1)

In [17]:
vgg16 = torchvision.models.vgg16(num_classes=300)

In [18]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
vgg16.avgpool

AdaptiveAvgPool2d(output_size=(7, 7))

In [13]:
vgg16.classifier[6] = nn.Linear(4096, 200)

In [14]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [19]:
7168/7

1024.0